In [13]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
from tensorflow.contrib import rnn
import random
#txt = pd.read_excel('数据比较好的.xlsx')
txt = pd.read_excel('guo(new_3w).xlsx').drop('空白字段',axis = 1)
Y = txt['标签']
'''
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 
list_des = txt['商品描述'].tolist()
X = []
for i in range(len(list_des)):
    des = list_des[i].replace(' ','')
    seg_list = jieba.lcut(des)
    word_list = [] 
    for seg in seg_list:
        if seg not in stopwords:
            word_list.append(seg)
    X.append((' ').join(word_list))
txt['商品描述'] = pd.Series(X) '''

"\njieba.load_userdict(Y)\nstopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] \nlist_des = txt['商品描述'].tolist()\nX = []\nfor i in range(len(list_des)):\n    des = list_des[i].replace(' ','')\n    seg_list = jieba.lcut(des)\n    word_list = [] \n    for seg in seg_list:\n        if seg not in stopwords:\n            word_list.append(seg)\n    X.append((' ').join(word_list))\ntxt['商品描述'] = pd.Series(X) "

In [14]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [15]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=10:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

数据量：21202 分类数目：511


In [16]:
count = CountVectorizer(min_df=5)
frematrix = count.fit_transform(frame['商品描述'])
tfidf = TfidfVectorizer().fit_transform(frame['商品描述'])

In [5]:
lr = LogisticRegression()
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.1)
lr.fit(X_train,Y_train)
pre = lr.predict(X_test)
np.mean(pre == Y_test)

0.851013672795851

In [17]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)
from gensim.models import word2vec
#sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(word_txt,size = 100,window = 2000,min_count = 3)

In [7]:
word2vec_matrix = []
for line in range(frame.shape[0]):
    vector = []
    num = 0
    words = frame['商品描述'][line].split()
    for word in words:
        try:
            temp = model[word]
        except:
            continue 
        else:
            vector += list(temp)
            num += 1
    vector = vector + [0]*100*(15 - num)
    word2vec_matrix.append(vector)

c:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [8]:
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
inputs = hstack((csr_matrix(np.array(word2vec_matrix)),frematrix)) #返回来的是coo_matrix ，不能进行索引，切片什么的
inputs_x = inputs.tocsr()#转化为 csr_matrix

In [9]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame['标签'])

In [10]:
input_size = 100
timestep_size = 15 
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.001
fre_shape = frematrix.shape[1]
add_size = frematrix.shape[1] + np.shape(word2vec_matrix)[1]

with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    _X = tf.placeholder(tf.float32,[None,1500])
    fre = tf.placeholder(tf.float32,[None,fre_shape])
    y = tf.placeholder(tf.int64,[None])
    x = tf.reshape(_X,[-1,timestep_size,input_size])
with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCellltiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)
with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
with tf.name_scope("output"):
    x = tf.unstack(x, 15, axis=1)
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
with tf.name_scope('concat'):
    output = tf.concat([outputs[-1],fre],1)
with tf.name_scope("result"):
    w = tf.Variable(tf.random_uniform([2 * hidden_size + fre_shape, class_num], -1.0, 1.0), name='W')
    b = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
    logits = tf.nn.xw_plus_b(output, w, b)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(inputs_x,y_label,test_size = 0.1)

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(10001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(X_train.shape[0]))
            Iter += 1
        temp_x = csr_matrix((_batch_size,add_size),dtype=float)
        temp_y = []
        for k in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x[k] = X_train[listnum[j]]
            #temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        #print(temp_x)
        if (i)%50 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test[:,:1500].toarray(),fre:X_test[:,1500:].toarray(), y: Y_test, 
                                                           keep_prob: 1.0, batch_size:X_test.shape[0]})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x[:,:1500].toarray(),fre:temp_x[:,1500:].toarray(),y:temp_y,
                                     keep_prob: 0.3, batch_size: _batch_size})

c:\python3\lib\site-packages\scipy\sparse\compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Iter 1: step 1, training accuracy 0.0018859
Iter 3: step 51, training accuracy 0.576615
Iter 6: step 101, training accuracy 0.685526
Iter 8: step 151, training accuracy 0.746818
Iter 11: step 201, training accuracy 0.789722
Iter 14: step 251, training accuracy 0.815653
Iter 16: step 301, training accuracy 0.835455
Iter 19: step 351, training accuracy 0.851014
Iter 22: step 401, training accuracy 0.860443
Iter 24: step 451, training accuracy 0.868458
Iter 27: step 501, training accuracy 0.871287
Iter 29: step 551, training accuracy 0.874587
Iter 32: step 601, training accuracy 0.880245
Iter 35: step 651, training accuracy 0.880717
Iter 37: step 701, training accuracy 0.880245


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf 
a = tf.Variable([[1,2],[3,4]])
b = tf.Variable([[5,6],[7,8]])
c = tf.concat([a,b],1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(c)
